In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import warnings
warnings.filterwarnings('ignore')

# Load dataset
# Load dataset
fertilizer = pd.read_csv('F:\crop and fertilizer recomendation system\data set\Fertilizer Prediction.csv')
fertilizer['Fertilizer Name'] = fertilizer['Fertilizer Name'].replace('10/26/2626', '10-26-26').str.strip().str.lower()

<>:19: SyntaxWarning: invalid escape sequence '\c'
<>:19: SyntaxWarning: invalid escape sequence '\c'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_15140\369603143.py:19: SyntaxWarning: invalid escape sequence '\c'
  fertilizer = pd.read_csv('F:\crop and fertilizer recomendation system\data set\Fertilizer Prediction.csv')


ENCODE 

In [2]:
# Encode categorical features
soil_encoder = LabelEncoder()
crop_encoder = LabelEncoder()
fertilizer['Soil Type'] = soil_encoder.fit_transform(fertilizer['Soil Type'])
fertilizer['Crop Type'] = crop_encoder.fit_transform(fertilizer['Crop Type'])
joblib.dump(soil_encoder, 'soil_encoder.pkl')
joblib.dump(crop_encoder, 'crop_encoder.pkl')


['crop_encoder.pkl']

In [3]:
# Features and target
features = ['Temparature', 'Humidity ', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous']
target = 'Fertilizer Name'
X = fertilizer[features]
y = fertilizer[target]

In [4]:

# Encode target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
joblib.dump(label_encoder, 'label_encoder_fert.pkl')

['label_encoder_fert.pkl']

SPLIT DATA

In [5]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [6]:

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, 'fertilizer_scaler.pkl')
print("Scaler feature names:", scaler.feature_names_in_)

Scaler feature names: ['Temparature' 'Humidity ' 'Moisture' 'Soil Type' 'Crop Type' 'Nitrogen'
 'Potassium' 'Phosphorous']


MODEL INITIALIZATION

In [7]:
# Initialize models
rf_model = RandomForestClassifier(random_state=100)
knn_model = KNeighborsClassifier(n_neighbors=5)
nb_model = GaussianNB()
svm_model = SVC(probability=True, random_state=100)
xgb_model = XGBClassifier(random_state=100, eval_metric='mlogloss')


In [8]:
# Voting Classifier
voting_clf_fert = VotingClassifier(
    estimators=[('rf', rf_model), ('knn', knn_model), ('nb', nb_model), ('svm', svm_model), ('xgb', xgb_model)],
    voting='soft'
)

In [9]:
# Train
voting_clf_fert.fit(X_train_scaled, y_train)
joblib.dump(voting_clf_fert, 'fertilizer_recommendation_model.pkl')

['fertilizer_recommendation_model.pkl']

In [10]:
# Predict and decode
y_pred_fert = voting_clf_fert.predict(X_test_scaled)
y_pred_fert_decoded = label_encoder.inverse_transform(y_pred_fert)
y_test_decoded = label_encoder.inverse_transform(y_test)


In [11]:
# Evaluate
accuracy = accuracy_score(y_test_decoded, y_pred_fert_decoded)
print("Voting Classifier Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test_decoded, y_pred_fert_decoded))

Voting Classifier Accuracy: 1.0

Classification Report:
              precision    recall  f1-score   support

    10-26-26       1.00      1.00      1.00         2
    14-35-14       1.00      1.00      1.00         1
       20-20       1.00      1.00      1.00         1
       28-28       1.00      1.00      1.00         5
         dap       1.00      1.00      1.00         5
        urea       1.00      1.00      1.00         6

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [12]:
# Fertilizer details
fertilizer_details = {
    'urea': {'full_name': 'Urea (Carbamide)', 'npk': '46-0-0', 'explanation': 'High-nitrogen fertilizer (46% Nitrogen). Used for crops needing nitrogen boost, like Paddy, Wheat, and Cotton. Promotes leaf and stem growth.'},
    'dap': {'full_name': 'Diammonium Phosphate', 'npk': '18-46-0', 'explanation': 'Rich in Phosphorus (46%) and Nitrogen (18%). Ideal for root development and early growth. Suits crops like Sugarcane, Ground Nuts, and Pulses.'},
    '14-35-14': {'full_name': 'NPK 14-35-14', 'npk': '14-35-14', 'explanation': 'Balanced fertilizer with Nitrogen (14%), Phosphorus (35%), and Potassium (14%). Supports flowering and fruiting. Great for Cotton, Sugarcane, and Oil seeds.'},
    '28-28': {'full_name': 'NPK 28-28-0', 'npk': '28-28-0', 'explanation': 'Equal Nitrogen and Phosphorus (28% each), no Potassium. Boosts vegetative growth and root strength. Common for Paddy, Maize, and Ground Nuts.'},
    '17-17-17': {'full_name': 'NPK 17-17-17', 'npk': '17-17-17', 'explanation': 'Fully balanced fertilizer (17% each of N, P, K). Versatile for all growth stages. Used for Sugarcane, Cotton, and Barley.'},
    '20-20': {'full_name': 'NPK 20-20-0', 'npk': '20-20-0', 'explanation': 'Balanced Nitrogen and Phosphorus (20% each). Supports early growth and root development. Ideal for Millets, Pulses, and Oil seeds.'},
    '10-26-26': {'full_name': 'NPK 10-26-26', 'npk': '10-26-26', 'explanation': 'Balanced fertilizer with low Nitrogen (10%), high Phosphorus (26%), and Potassium (26%). Supports flowering and fruiting. Common for Pulses and Barley.'}
}

In [13]:
# Recommend fertilizer function
def recommend_fertilizer(Temparature, Humidity, Moisture, Soil_Type, Crop_Type, Nitrogen, Potassium, Phosphorous):
    try:
        # Load scaler, encoders, and model
        sc = joblib.load('fertilizer_scaler.pkl')
        soil_encoder = joblib.load('soil_encoder.pkl')
        crop_encoder = joblib.load('crop_encoder.pkl')
        model_fert = joblib.load('fertilizer_recommendation_model.pkl')

        # Validate numerical inputs
        if not (0 <= Temparature <= 42):
            return {'name': 'Error', 'full_name': 'Error', 'npk': 'N/A', 'explanation': 'Temperature must be between 0 and 42.'}
        if not (25 <= Humidity <= 38):
            return {'name': 'Error', 'full_name': 'Error', 'npk': 'N/A', 'explanation': 'Humidity must be between 25 and 38.'}
        if not (50 <= Moisture <= 72):
            return {'name': 'Error', 'full_name': 'Error', 'npk': 'N/A', 'explanation': 'Moisture must be between 50 and 72.'}
        if not (4 <= Nitrogen <= 65):
            return {'name': 'Error', 'full_name': 'Error', 'npk': 'N/A', 'explanation': 'Nitrogen must be between 4 and 65.'}
        if not (0 <= Potassium <= 42):
            return {'name': 'Error', 'full_name': 'Error', 'npk': 'N/A', 'explanation': 'Potassium must be between 0 and 42.'}
        if not (0 <= Phosphorous <= 19):
            return {'name': 'Error', 'full_name': 'Error', 'npk': 'N/A', 'explanation': 'Phosphorous must be between 0 and 19.'}

        # Encode categorical inputs
        if Soil_Type not in soil_encoder.classes_:
            return {'name': 'Error', 'full_name': 'Error', 'npk': 'N/A', 'explanation': f'Invalid Soil Type. Choose from {list(soil_encoder.classes_)}.'}
        if Crop_Type not in crop_encoder.classes_:
            return {'name': 'Error', 'full_name': 'Error', 'npk': 'N/A', 'explanation': f'Invalid Crop Type. Choose from {list(crop_encoder.classes_)}.'}
        
        soil_encoded = soil_encoder.transform([Soil_Type])[0]
        crop_encoded = crop_encoder.transform([Crop_Type])[0]

        # Prepare input array (8 features)
        features = np.array([[Temparature, Humidity, Moisture, soil_encoded, crop_encoded, Nitrogen, Potassium, Phosphorous]])
        columns = ['Temparature', 'Humidity ', 'Moisture', 'Soil Type', 'Crop Type', 'Nitrogen', 'Potassium', 'Phosphorous']
        input_df = pd.DataFrame(features, columns=columns)

        # Scale features
        transformed_features = sc.transform(input_df)

        # Predict
        prediction = model_fert.predict(transformed_features)[0]
        fertilizer = label_encoder.inverse_transform([prediction])[0]

        # Get details
        details = fertilizer_details.get(fertilizer, {
            'full_name': fertilizer,
            'npk': 'Unknown',
            'explanation': 'No details available.'
        })

        return {
            'name': fertilizer,
            'full_name': details['full_name'],
            'npk': details['npk'],
            'explanation': details['explanation']
        }
    except Exception as e:
        return {
            'name': 'Error',
            'full_name': 'Error',
            'npk': 'N/A',
            'explanation': f'Error in prediction: {str(e)}'
        }



In [14]:
# Test function
result = recommend_fertilizer(26, 30, 60, 'Sandy', 'Maize', 37, 20, 19)
print("\nRecommendation Result:")
print(result)


Recommendation Result:
{'name': '10-26-26', 'full_name': 'NPK 10-26-26', 'npk': '10-26-26', 'explanation': 'Balanced fertilizer with low Nitrogen (10%), high Phosphorus (26%), and Potassium (26%). Supports flowering and fruiting. Common for Pulses and Barley.'}


In [15]:
import os
print(os.listdir('.'))

['1filtered_crop_recommendation.csv', 'app.py', 'app2.py', 'crop.ipynb', 'crop_encoder.pkl', 'crop_recommendation_model.pkl', 'crop_scaler.pkl', 'data set', 'fertilizer.ipynb', 'fertilizer2.ipynb', 'fertilizer_recommendation_model.pkl', 'fertilizer_scaler.pkl', 'filtered_crop_recommendation.csv', 'label_encoder_fert.pkl', 'scaler.pkl', 'scaler_crop.pkl', 'signup.py', 'soil_encoder.pkl', 'tempCodeRunnerFile.py']
